![](http://kodcu.com/wp/wp-content/uploads/2014/06/mllib.png)

![](http://img.scoop.it/_VP0qLV-jYbp2cFF4H4k4jl72eJkfbmt4t8yenImKBVvK0kTmF0xjctABnaLJIm9)

In [1]:
from pyspark.mllib import *

![](https://media.giphy.com/media/b7FNjKdGXEFos/giphy.gif)

---
MLlib Overview
---

MLlib is a Spark subproject (originally started in AMPlab in Berkeley) providing machine learning primitives that are "production" ready. This is an incredible benefit to the data science community.

Widespread adoption and support - 80+ contributors from various organizations. 
 
__Remember__: You do not _need_ to use MLlib to do large scale machine learning. 

Depending on the nature of the problem there are other ways to do large scale machine learning:

1. Use a large single-node machine learning library (_i.e._ `scikit-learn`) on big EC2 instances.
2. Distribute embarassingly parallel problems as a grid search  (relatively common). 
3. Use MapReduce to design a model composed of (several) multiple map-reduce steps. This is essentially what MLLib does for you, in so many words.

----
[MLlib Algorithms](http://spark.apache.org/mllib/)
----
- logistic regression and linear support vector machine (SVM)
- classification and regression tree
- random forest and gradient-boosted trees
- recommendation via alternating least squares (ALS)
- clustering via k-means, bisecting k-means, Gaussian mixtures (GMM), and power iteration clustering
- topic modeling via latent Dirichlet allocation (LDA)
- survival analysis via accelerated failure time model
- singular value decomposition (SVD) and QR decomposition
- principal component analysis (PCA)
- linear regression with L1, L2, and elastic-net regularization
- isotonic regression
- multinomial/binomial naive Bayes
- frequent itemset mining via FP-growth and association rules
- sequential pattern mining via PrefixSpan
- summary statistics and hypothesis testing
- feature transformations
- model evaluation and hyper-parameter tuning



As wondrous as the implementation of these models are, their very presence will not save you from a bad choice of model, poorly-manicured data (bad pipelines/feature engineering), or a lack of signal. You still have to be a data scientist.

---
Points to Ponder
---

<details><summary>
What types of algorithms are not in Spark? Why do you think that is?
</summary>
Deep Learning. [They are working on it](http://arxiv.org/abs/1511.06051)
</details>

-----
Spark Statistics
-----

In [2]:
import numpy as np

from pyspark.mllib.stat import Statistics

In [3]:
import pyspark
# Creating SparkContext not necessary,
# because of launching notebook with pyspark 
# `$ IPYTHON_OPTS="notebook" pyspark` 
sc = pyspark.SparkContext()
# sc = pyspark.SparkContext() 
print sc

In [4]:
# Let's look at the what is available...
Statistics.

SyntaxError: invalid syntax (<ipython-input-4-12e05460d4e2>, line 2)

In [8]:
# Create a RDD of Vectors
mat = sc.parallelize([np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([3.0, 30.0, 300.0])])  
mat.collect()

[array([   1.,   10.,  100.]),
 array([   2.,   20.,  200.]),
 array([   3.,   30.,  300.])]

In [9]:
# Compute column summary statistics.
summary = Statistics.colStats(mat)

In [10]:
# Let's look at the what is available...
summary.

SyntaxError: invalid syntax (<ipython-input-10-345574200ac9>, line 2)

In [11]:
print("The mean of each column: {}".format(summary.mean()))
print("The mean of each column: {}".format(summary.variance()))

The mean of each column: [   2.   20.  200.]
The mean of each clumn: [  1.00000000e+00   1.00000000e+02   1.00000000e+04]


---
Check for understanding
---
<p><details><summary>What common statistics are missing from this list? Why?</summary>
Median and other quantiles are missing.  
<br>
Why? Because they are non-associative, thus harder to parrelleize
</details></p>

# k-means Algorithm

The k-means algorithm

* Choose a number of clusters k
* Randomly assign each point to a cluster
* Repeat:
    * a\. For each of k clusters, compute cluster *centroid* by taking
mean vector of points in the cluster
    * b\. Assign each data point to cluster for which centroid is closest
(Euclidean)

...until clusters stop changing

[Source](https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/mllib-statistics.html)

![](images/kmeans.png)

![](images/clusters.png)

![](images/choose.png)

![](images/adjust.png)

![](images/spark_kmeans.png)

------
K-means in Spark 2.0
------

In [12]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import KMeans

In [13]:
data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([1.0, 1.0]),),
         (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]
df = sqlContext.createDataFrame(data, ["features"])

In [14]:
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(df)

[Source]( https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/api/python/pyspark.ml.html?highlight=kmeans#pyspark.ml.clustering.KMeans )

---
Points to Ponder
---

<details><summary>
1) Why is K-means in Spark's MLlib? A better reason to say this is: Why bother to put K-means in MLLib in the first place?
</summary>
K-means requires a complete pass over the data every iteration. Spark is  efficient for that specification and so makes the implementation of the algorithm worthwhile. In general, calculations that are of quadratic order
</details>



-----
Recommendations via matrix completion
-----

![](images/recommend.png)

Collaborative Filtering
---

Goal: predict users’ movie ratings based on past ratings of other movies

$$
R = \left( \begin{array}{ccccccc}
1 & ? & ? & 4 & 5 & ? & 3 \\
? & ? & 3 & 5 & ? & ? & 3 \\
5 & ? & 5 & ? & ? & ? & 1 \\
4 & ? & ? & ? & ? & 2 & ?\end{array} \right)
$$


<details><summary>
What are the challenges of large scale Collaborative Filtering?
</summary>
Challenges: <br>
- Defining similarity that scales <br>
- Dimensionality (Millions of Users / Items) <br>
- Sparsity, most users has not seen most items <br>
<br>
</details>



<details><summary>
Why Spark for collaborative filtering? Why not Map-Reduce?
</summary>
a) In-memory processing of the data  
b) Real-time data processing capabilities
c) Easier implementation
</details>

# Singular Value Decomposition (SVD)

Every matrix has a *unique* decomposition in the following form:




![SVD](images/SVD.png)

where
* *U* is column orthogonal: *U<sup>T</sup>U = I*
* *V* is column orthogonal: *V<sup>T</sup>V = I*
* *Sigma* is a diagonal matrix of positive values, where the diagonal is ordered in decreasing order

We can reduce the dimensions by sending the smaller of the diagonals to 0.


# SVD for topic analysis

We can use SVD to determine what we call ***latent features***. This will be best demonstrated with an example.

### Example

Let's look at users ratings of different movies. The ratings are from 1-5. A rating of 0 means the user hasn't watched the movie.

|           | Matrix | Alien | Serenity | Casablanca | Amelie |
| --------- | ------ | ----- | -------- | ---------- | ------ |
| **Alice** |      1 |     1 |        1 |          0 |      0 |
|   **Bob** |      3 |     3 |        3 |          0 |      0 |
| **Cindy** |      4 |     4 |        4 |          0 |      0 |
|   **Dan** |      5 |     5 |        5 |          0 |      0 |
| **Emily** |      0 |     2 |        0 |          4 |      4 |
| **Frank** |      0 |     0 |        0 |          5 |      5 |
|  **Greg** |      0 |     1 |        0 |          2 |      2 |

Note that the first three movies (Matrix, Alien, Serenity) are Sci-fi movies and the last two (Casablanca, Amelie) are Romance. We will be able to mathematically pull out these topics!

Let's do the computation with Python.


In [15]:

from numpy.linalg import svd

M = np.array([[1, 1, 1, 0, 0],
              [3, 3, 3, 0, 0],
              [4, 4, 4, 0, 0],
              [5, 5, 5, 0, 0],
              [0, 2, 0, 4, 4],
              [0, 0, 0, 5, 5],
              [0, 1, 0, 2, 2]])

u, e, v = svd(M)
print M
print "="
print np.around(u, 2)
print np.around(e, 2)
print np.around(v, 2)

[[1 1 1 0 0]
 [3 3 3 0 0]
 [4 4 4 0 0]
 [5 5 5 0 0]
 [0 2 0 4 4]
 [0 0 0 5 5]
 [0 1 0 2 2]]
=
[[-0.14  0.02  0.01  0.99 -0.   -0.    0.  ]
 [-0.41  0.07  0.03 -0.06 -0.89  0.19  0.  ]
 [-0.55  0.09  0.04 -0.08  0.42  0.71  0.  ]
 [-0.69  0.12  0.05 -0.1   0.19 -0.68  0.  ]
 [-0.15 -0.59 -0.65 -0.    0.   -0.   -0.45]
 [-0.07 -0.73  0.68  0.   -0.    0.    0.  ]
 [-0.08 -0.3  -0.33 -0.   -0.   -0.    0.89]]
[ 12.48   9.51   1.35   0.     0.  ]
[[-0.56 -0.59 -0.56 -0.09 -0.09]
 [ 0.13 -0.03  0.13 -0.7  -0.7 ]
 [ 0.41 -0.8   0.41  0.09  0.09]
 [-0.71  0.    0.71 -0.    0.  ]
 [-0.    0.   -0.    0.71 -0.71]]




Here's the results we get:

![ ](images/svd_example.png)

Note that the last two singular values are 0, so we can drop them. Note that these values are 0 because the rank of our original matrix is 3.

![SVD Example](images/svd_example2.png)

You can see the two topics fall out:

1. Science Fiction
    * First singular value (12.4)
    * First column of the *U* matrix (note that the first four users have large values here)
    * First row of the *V* matrix (note that the first three movies have large values here)
2. Romance
    * Second singular value (9.5)
    * Second column of the *U* matrix (note that the last three users have large values here)
    * Second row of the *V* matrix (note that the last two movies have large values here)

*U* is the ***user-to-topic*** matrix and *V* is the ***movie-to-topic*** matrix.

The third singular value is relatively small, so we can exclude it with little loss of data. Let's try doing that and reconstruct our matrix!


![SVD Example](images/svd_example3.png)


# Alternating Least squares

![](images/solution.png)

## Alternating Least Squares  
![ALS](https://databricks.com/wp-content/uploads/2014/07/als-illustration.png)  
1. Start with random $U_1$, $V_1$
2. (LU decomposition) Solve for $U_2$ to minimize $||R – U_2V_1^T||$ subject to $V_1$
3. (LU decomposition) Solve for $V_2$ to minimize $||R – U_2V_2^T||$ subject to $U_2$
4. Repeat until convergence

-----
ALS in Spark 2.0
-----

In [16]:
# from pyspark.mllib.recommendation import ALS

In [17]:
# from pyspark import SparkConf, SparkContext

In [18]:
# df = sqlContext.createDataFrame([(0, 1, 4.0), (1, 1, 5.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],
                           ["user", "item", "rating"])
# display(df)

IndentationError: unexpected indent (<ipython-input-18-7d5d428850d2>, line 2)

In [19]:
# df.show()

In [20]:
# als = ALS() 
# model = als.train(df, rank=10)

In [21]:
# user = 0
# item = 2
# model.predict(user, item)

[Source](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.recommendation)

---
ALS vs. SVD for recommendation engines
---

Alternating least squares (ALS) is flexible but less precise.  "Approximate" means minimizing some squared-error difference with the input A, but, here you can customize exactly what is considered in the loss function. For example you can ignore missing values (crucial) or weight different values differently.

Singular value decomposition (SVD) is a decomposition that gives more guarantees about its factorization. The SVD is relatively more computationally expensive and harder to parallelize. There is also not a good way to deal with missing values or weighting; you need to assume that in your sparse input, missing values are equal to a mean value 0. 

[Source](https://www.quora.com/What-is-the-difference-between-SVD-and-matrix-factorization-in-context-of-recommendation-engine)

---
Summary
----

- MLlib is Spark's machine learning framework. 
- A bunch of super smart people are porting even more algorithms to work in distributed, lazy-exectution way.+987
- MLlib has the greatest hits of machine learning:
    - K-means for clustering
    - ALS for Collaborative Filtering

---
Bonus Materials
---

---
MLlib Data Types
---

[Docs](http://spark.apache.org/docs/latest/mllib-data-types.html)

`Local vector`: can be dense or sparse

A dense vector is backed by a double array representing its entry values.

While a sparse vector is backed by two parallel arrays: indices and values. 

For example, a vector (1.0, 0.0, 3.0) can be represented in 

| Dense | Sparse |  
|:-------:|:------:|
| [1.0, 0.0, 3.0]  | (3, [0, 2], [1.0, 3.0]) |


In [22]:
from pyspark.mllib.linalg import Vectors

# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])

In [23]:
sv1

SparseVector(3, {0: 1.0, 2: 3.0})

`LabeledPoint`

A labeled point is a local vector, either dense or sparse, associated with a label/response for supervised learning algorithms

In [24]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Create a labeled point with a positive label and a dense feature vector.
pos = LabeledPoint(1.0, [1.0, 0.0, 3.0])

# Create a labeled point with a negative label and a sparse feature vector.
neg = LabeledPoint(0.0, SparseVector(3, [0, 2], [1.0, 3.0]))

Local matrix

Take a _wild guess_...

> A local matrix has integer-typed row and column indices and double-typed values, stored on a single machine. 

Distributed matrix

> A distributed matrix has long-typed row and column indices and double-typed values, stored distributively in one or more RDDs. 

It is very important to choose the right format to store large and distributed matrices. Converting a distributed matrix to a different format may require a global shuffle, which is quite expensive. Three types of distributed matrices have been implemented so far.

In [25]:
from pyspark.mllib.linalg.distributed import RowMatrix

# Create an RDD of vectors.
rows = sc.parallelize([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

# Create a RowMatrix from an RDD of vectors.
mat = RowMatrix(rows)

# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3

# Get the rows as an RDD of vectors again.
rowsRDD = mat.rows

In [26]:
rowsRDD.

SyntaxError: invalid syntax (<ipython-input-26-ff1cbc6e9998>, line 1)

---
Check for understanding
---
<details><summary>
Which data type would you use for random forests?
</summary>
`LabeledPoint` <br>
<br>
Random forests is a supervised learning algorithm. It requires labels in order to classify.
</details>